In [ ]:
# Imports
import random
import pandas as pd
import math
import os
from datetime import datetime
from supabase import create_client

## Database Connection

In [ ]:
supabase_url = os.environ.get('URL_J2D')
supabase_key = os.environ.get('SUPA_KEY_J2D')

supabase = create_client(supabase_url, supabase_key)

## Monuments Positions

In [ ]:
# Reading file
monu_pos = pd.read_csv("puntos_bnc.csv", 
                       names=['id', 'title', 'lat', 'lon', 'description', 'url'], 
                       header=0)

In [ ]:
# For updating rows
monu_pos_dict = monu_pos.to_dict('records')

Monuments DB Update

In [ ]:
# Update monument's data 
for row in monu_pos_dict:
   response = supabase.table("monument").upsert(row, on_conflict="id").execute()

## Initial User Positions

In [ ]:
a = (41.375715, 2.090579) #(41.339365, 2.101318)
b = (41.452426, 2.226904) #(41.448274, 2.198499)

def generate_random_point(a, b):

    lat = random.uniform(a[0], b[0])
    lon = random.uniform(a[1], b[1])

    return {'lat': lat, 'lon': lon}

user_init_pos_df = pd.DataFrame([generate_random_point(a, b) for _ in range(500)])
user_init_pos_df.reset_index(inplace=True)
user_init_pos_df.rename(columns={'index': 'id'}, inplace=True)

# Converting DataFrame to a list of dictionaries
user_init_pos = user_init_pos_df.to_dict('records')

Users Initial Position DB Update

In [ ]:
# Update user's data for initial positions
for row in user_init_pos:
   response = supabase.table("user").upsert(row, on_conflict="id").execute()

## Generate Random Movements

In [ ]:
#NW = (41.458157, 2.182716) #NE, (41.451398, 2.182092)
#SE = (41.353727, 2.061600) #SW, (41.365285, 2.076672)
#NE = (41.387520, 2.082303) #NW, (41.305722, 2.136167)
#SW = (41.412755, 2.223006) #SE, (41.410313, 2.224887)

a = (41.375715, 2.090579) #(41.339365, 2.101318)
b = (41.452426, 2.226904) #(41.448274, 2.198499)

# Run each 10 seconds to deply in databse
def is_point_within_area(point, a, b):

    lat, lon = point
    return (a[0] <= lat <= b[0]) and (a[1] <= lon <= b[1])


def generate_new_point(current_position, distance=417):  # distance in meters
    # 1 degree latitude is approximately 111 kilometers
    # 1 degree longitude varies based on latitude, but will use an average for simplicity
    lat_dist = distance / 111000  
    lon_dist = distance / (111000 * math.cos(math.radians(current_position[0])))

    new_lat = current_position[0] + random.uniform(-lat_dist, lat_dist)
    new_lon = current_position[1] + random.uniform(-lon_dist, lon_dist)

    return (new_lat, new_lon) if is_point_within_area((new_lat, new_lon), a, b) else current_position


def simulate_movement(row):

    new_position = generate_new_point((row['lat'], row['lon']))
    return pd.Series({'lat': new_position[0], 'lon': new_position[1]})

In [ ]:
user_mov_df = user_init_pos_df.apply(simulate_movement, axis=1).reset_index()
user_mov_df.columns = ['id', 'lat', 'lon']
user_mov_pos = user_mov_df.to_dict('records')

Users Random Position DB Update

In [ ]:
# Update data with random movements
for row in user_mov_pos:
   response = supabase.table("user").upsert(row, on_conflict="id").execute()

### Saving User Track

Saving user historical data for historical analytics putrposes

In [ ]:
# Insert user's data for trajectory
for row in user_mov_pos:
    trajectory_data = {
        'user_id': row['id'],
        'latitude': row['lat'],
        'longitude': row['lon'],
        'timestamp': datetime.now().isoformat()
    }
    response = supabase.table("user_trajectory").insert(trajectory_data).execute()

### Real Open BCN Data Processing

In [ ]:
# Columns to get from the data source
cols = ['name', 'geo_epgs_4326_x', 'geo_epgs_4326_y']

In [ ]:
open_bcn_df = pd.read_csv('opendatabcn_pics-csv.csv', 
                 encoding='ISO-8859-1',
                 header=None,
                 delimiter=',').reset_index()

In [ ]:
# Split the single column into multiple columns
open_bcn_df = open_bcn_df[0].str.split(',', expand=True)

In [ ]:
# Optionally set the first row as headers if your CSV doesn't have headers
open_bcn_df.columns = open_bcn_df.iloc[0]
open_bcn_df = open_bcn_df.drop(open_bcn_df.index[0])

In [ ]:
# Filter by columns of interest
open_bcn_df = open_bcn_df[['name', 'geo_epgs_4326_x', 'geo_epgs_4326_y']].reset_index()
open_bcn_df.columns = ['id', 'title', 'lat', 'lon']

In [ ]:
open_bcn_df.info()

Creating table for the Open Data BCN

In [ ]:
# To iterate
open_bcn_dict = open_bcn_df.to_dict('records')

In [ ]:
# Update monument's data from OpenDataBCN 
for row in open_bcn_dict:
   response = supabase.table("monument_openbcn").upsert(row, on_conflict="id").execute()